##Урок 4. Uplift-моделирование

In [8]:
#!pip install scikit-uplift==0.2.0

####1.
 скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [1]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k
from sklift.models import SoloModel
from sklift.models import TwoModels
from sklift.models import ClassTransformation
from sklearn.ensemble import RandomForestClassifier

In [49]:
df = pd.read_csv("data.csv")
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


###2. 
там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [50]:
df = df.rename(columns={'conversion': 'target', 'offer': 'treatment'}) 
df.head(1)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0


In [51]:
df['treatment'].unique()

array(['Buy One Get One', 'No Offer', 'Discount'], dtype=object)

In [52]:
df.loc[df['treatment'] == 'Buy One Get One', 'treatment'] = 1
df.loc[df['treatment'] == 'Discount', 'treatment'] = 1
df.loc[df['treatment'] == 'No Offer', 'treatment'] = 0
df['treatment'].unique()

array([1, 0], dtype=object)

####4. 
сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [53]:
df['zip_code'].unique()

array(['Surburban', 'Rural', 'Urban'], dtype=object)

In [54]:
df = pd.get_dummies(df, columns=['zip_code'])
df.head(1)

,recency,history,used_discount,used_bogo,is_referral,channel,treatment,target,zip_code_Rural,zip_code_Surburban,zip_code_Urban
0,10,142.44,1,0,0,Phone,1,0,0,1,0


In [55]:
df['channel'].unique()

array(['Phone', 'Web', 'Multichannel'], dtype=object)

In [56]:
df = pd.get_dummies(df, columns=['channel'])
df.head(1)

,recency,history,used_discount,used_bogo,is_referral,treatment,target,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,10,142.44,1,0,0,1,0,0,1,0,0,1,0


####3.
 сделать разбиение набора данных не тренировочную и тестовую выборки

In [57]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='target'), df['target'], test_size=0.2, random_state=17)
treat_train = X_train['treatment']
treat_test = X_test['treatment']

####5.
 провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

In [58]:
models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': []
}

In [59]:
sm = SoloModel(RandomForestClassifier(n_estimators=300, max_depth=8, min_samples_leaf=7, random_state=42))
sm = sm.fit(X_train, y_train, treat_train)

uplift_sm = sm.predict(X_test)

sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)

models_results['approach'].append('SoloModel')
models_results['uplift@10%'].append(sm_score_10)
models_results['uplift@20%'].append(sm_score_20)

In [60]:
tm = TwoModels(
    estimator_trmnt=RandomForestClassifier(n_estimators=300, max_depth=8, min_samples_leaf=7, random_state=42), 
    estimator_ctrl=RandomForestClassifier(n_estimators=300, max_depth=8, min_samples_leaf=7, random_state=42), 
    method='vanilla'
)
tm = tm.fit(X_train, y_train, treat_train)

uplift_tm = tm.predict(X_test)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.2)

models_results['approach'].append('TwoModels')
models_results['uplift@10%'].append(tm_score_10)
models_results['uplift@20%'].append(tm_score_20)

In [61]:
ct = ClassTransformation(RandomForestClassifier(n_estimators=300, max_depth=8, min_samples_leaf=7, random_state=42))
ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_test)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

models_results['approach'].append('ClassTransformation')
models_results['uplift@10%'].append(ct_score_10)
models_results['uplift@20%'].append(ct_score_20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  


####6.
 в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

In [62]:
pd.DataFrame(data=models_results)

,approach,uplift@10%,uplift@20%
0,SoloModel,0.088924,0.086753
1,TwoModels,0.107695,0.077960
2,ClassTransformation,0.256719,0.196366
